In [1]:
import numpy as np 
# import matplotlib.pyplot as plt 
# import pandas as pd
import glob as glob
# import sys as sys
# import csv as csv
# import random as rand
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.utils import Sequence
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.utils import to_categorical
# from sklearn.preprocessing import LabelEncoder
# from sklearn import preprocessing
# from tensorflow.keras import layers
# from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
# # from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.layers import LayerNormalization, BatchNormalization, TimeDistributed
# from sklearn import metrics
# from keras import models, losses, optimizers, callbacks
# from keras import backend as K
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report
# import seaborn as sns
# import imageio.v3 as iio
import cv2 #opencv
# import webcolors
# import colorsys

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Load training data:

In [14]:
import cv2
import os

def load_images_from_folder(folder):
    images = []
    for filename in sorted(os.listdir(folder)):
        print(filename)
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images


def load_images_from_filenames(filenames):
    images = []
    sorted_filenames = sorted(filenames, key=lambda x: int(x.split('.')[0][6:]))
    for filename in sorted_filenames:
        print(filename)
        img = cv2.imread(filename)
        if img is not None:
            images.append(img)
    return images

In [15]:
# path="/content/drive/MyDrive/train"
filenames= glob.glob('train/*.png')
# X=load_images_from_folder(path)
X=load_images_from_filenames(filenames)
# print(len(X)) #2000
# print(X[0].shape) #(150, 500, 3)

train/0.png
train/1.png
train/2.png
train/3.png
train/4.png
train/5.png
train/6.png
train/7.png
train/8.png
train/9.png
train/10.png
train/11.png
train/12.png
train/13.png
train/14.png
train/15.png
train/16.png
train/17.png
train/18.png
train/19.png
train/20.png
train/21.png
train/22.png
train/23.png
train/24.png
train/25.png
train/26.png
train/27.png
train/28.png
train/29.png
train/30.png
train/31.png
train/32.png
train/33.png
train/34.png
train/35.png
train/36.png
train/37.png
train/38.png
train/39.png
train/40.png
train/41.png
train/42.png
train/43.png
train/44.png
train/45.png
train/46.png
train/47.png
train/48.png
train/49.png
train/50.png
train/51.png
train/52.png
train/53.png
train/54.png
train/55.png
train/56.png
train/57.png
train/58.png
train/59.png
train/60.png
train/61.png
train/62.png
train/63.png
train/64.png
train/65.png
train/66.png
train/67.png
train/68.png
train/69.png
train/70.png
train/71.png
train/72.png
train/73.png
train/74.png
train/75.png
train/76.png
train/77.

In [5]:
# from google.colab.patches import cv2_imshow

# cv2_imshow(X[0])

Convert to HSV:

In [6]:
def rgb_to_hsv(X_rgb):
  X_hsv=[]
  for img in X_rgb:
    X_hsv.append(cv2.cvtColor(img, cv2.COLOR_BGR2HSV))
  return X_hsv

In [7]:
X=rgb_to_hsv(X)

# cv2_imshow(X[0])

Erode for edge darkness and eliminate obfuscating lines:

In [8]:
kernel = np.ones((5, 5), np.uint8)
  
# Using cv2.erode() method 
img = cv2.erode(X[0], kernel) 
# cv2_imshow(img)

Remove background and convert to grayscale:

Segment the image into 3 images:

In [9]:
def processImage(img):
  #erode
  kernel = np.ones((5, 5), np.uint8)
  
  # Using cv2.erode() method 
  img = cv2.erode(img, kernel) 


  red = img[0][0][0]
  green = img[0][0][1]
  blue = img[0][0][2]

  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      for k in range(3):
        if k==0:
          if img[i][j][k] < red:
            img[i][j][k]=0
          else:
            img[i][j][k] = img[i][j][k]-red
        elif k==1:
          if img[i][j][k] < green:
            img[i][j][k]=0
          else:
            img[i][j][k] = img[i][j][k]-green
        else:
          if img[i][j][k] < blue:
            img[i][j][k]=0
          else:
            img[i][j][k] = img[i][j][k]-blue

  #Removing the background to white
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      for k in range(3):
        img[i][j][k]=255-img[i][j][k]
  
  grayimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  filename= 'grayimage.png'
  cv2.imwrite(filename,grayimg)

  for i in range(grayimg.shape[0]):
    for j in range(grayimg.shape[1]):
      if(grayimg[i][j]!=255):
        grayimg[i][j]=0
  
  p1start=-1
  p1end=-1
  sizeOfImage = int(500/3)+1
  for j in range(grayimg.shape[1]):
    sum=0
    for i in range(grayimg.shape[0]):
      if(grayimg[i][j]!=255):
        sum+=1

    if(p1start==-1 and sum>=20):
      p1start=j
    elif(p1start!=-1 and sum<20):
      p1end=j
      break

  # print(p1end,p1start)
  img0 = np.zeros((150,p1end+5-p1start+5))
  img0 = grayimg[:,p1start-4:p1end+5]

  toadd = sizeOfImage - img0.shape[1]
  righthalf = int(toadd/2)
  lefthalf = int(toadd-righthalf)

  img0 = np.append(img0,255*np.ones((150,righthalf)),axis=1)
  img0 = np.append(255*np.ones((150,lefthalf)),img0,axis=1)

  p2start=-1
  p2end=-1
  for j in range(p1end+1,grayimg.shape[1]):
    sum=0
    for i in range(grayimg.shape[0]):
      if(grayimg[i][j]!=255):
        sum+=1
    if(p2start==-1 and sum>=20):
      p2start=j
    elif(p2start!=-1 and sum<20):
      p2end=j
      break

  img1 = np.zeros((150,p2end+5-p2start+5))
  img1 = grayimg[:,p2start-4:p2end+5]

  toadd = sizeOfImage - img1.shape[1]
  righthalf = int(toadd/2)
  lefthalf = int(toadd-righthalf)

  img1 = np.append(img1,255*np.ones((150,righthalf)),axis=1)
  img1 = np.append(255*np.ones((150,lefthalf)),img1,axis=1)

  p3start=-1
  p3end=-1
  for j in range(p2end+1,grayimg.shape[1]):
    sum=0
    for i in range(grayimg.shape[0]):
      if(grayimg[i][j]!=255):
        sum+=1
    if(p3start==-1 and sum>=20):
      p3start=j
    elif(p3start!=-1 and sum<20):
      p3end=j
      break

  img2 = np.zeros((150,p3end+5-p3start+5))
  img2 = grayimg[:,p3start-4:p3end+5]

  toadd = sizeOfImage - img2.shape[1]
  righthalf = int(toadd/2)
  lefthalf = int(toadd-righthalf)

  img2 = np.append(img2,255*np.ones((150,righthalf)),axis=1)
  img2 = np.append(255*np.ones((150,lefthalf)),img2,axis=1)

  for i in range(img0.shape[0]):
    for j in range(img0.shape[1]):
      img0[i][j]=255-img0[i][j]


  for i in range(img1.shape[0]):
    for j in range(img1.shape[1]):
      img1[i][j]=255-img1[i][j]
  
  
  for i in range(img2.shape[0]):
    for j in range(img2.shape[1]):
      img2[i][j]=255-img2[i][j]

  return img0,img1,img2

In [10]:
kernel = np.ones((10, 10), np.uint8)

Alter the dataset and add labels to each image (separately):

In [12]:
import pandas

labels=[]

df=pandas.read_csv("train/labels.txt",header=None)

for i in range (0,df.shape[0]):
  labels.append(df[0][i])
  labels.append(df[1][i])
  labels.append(df[2][i])

print(labels)

ct=0
for img in X:
  print(ct)
  
  try:
    l1,l2,l3=processImage(img)

  except:
    print("Error in image "+ str(ct/3)+".png")
      
  else:
    cv2.imwrite(os.path.join("new_data/" , str(ct)+".png"),l1)
    ct+=1
    cv2.imwrite(os.path.join("new_data/" , str(ct)+".png"),l2)
    ct+=1
    cv2.imwrite(os.path.join("new_data/" , str(ct)+".png"),l3)
    ct+=1


['ZETA', 'EPSILON', 'LAMDA', 'DELTA', 'ALPHA', 'EPSILON', 'RHO', 'PHI', 'PI', 'TAU', 'XI', 'PSI', 'PI', 'OMICRON', 'IOTA', 'DELTA', 'ALPHA', 'CHI', 'ZETA', 'NU', 'OMEGA', 'OMEGA', 'PHI', 'ZETA', 'ALPHA', 'NU', 'CHI', 'OMICRON', 'ETA', 'LAMDA', 'KAPPA', 'DELTA', 'PI', 'KAPPA', 'MU', 'OMICRON', 'ETA', 'THETA', 'XI', 'TAU', 'RHO', 'EPSILON', 'RHO', 'TAU', 'KAPPA', 'BETA', 'OMICRON', 'ETA', 'LAMDA', 'EPSILON', 'PI', 'IOTA', 'EPSILON', 'OMEGA', 'RHO', 'ALPHA', 'UPSILON', 'ALPHA', 'EPSILON', 'PSI', 'XI', 'OMICRON', 'LAMDA', 'PI', 'ZETA', 'GAMMA', 'PSI', 'XI', 'ALPHA', 'ZETA', 'XI', 'NU', 'PI', 'CHI', 'THETA', 'ETA', 'IOTA', 'ETA', 'THETA', 'ZETA', 'ALPHA', 'OMICRON', 'IOTA', 'PHI', 'NU', 'XI', 'GAMMA', 'DELTA', 'PI', 'SIGMA', 'PSI', 'PSI', 'PHI', 'KAPPA', 'OMICRON', 'THETA', 'IOTA', 'KAPPA', 'ZETA', 'XI', 'GAMMA', 'PSI', 'CHI', 'IOTA', 'ETA', 'XI', 'NU', 'ZETA', 'ZETA', 'OMICRON', 'UPSILON', 'MU', 'TAU', 'TAU', 'MU', 'DELTA', 'CHI', 'GAMMA', 'BETA', 'KAPPA', 'OMEGA', 'IOTA', 'PI', 'KAPPA', '

In [ ]:
labels=[]

df=pandas.read_csv("train/labels.txt",header=None)

for i in range (0,df.shape[0]):
  labels.append(df[0][i])
  labels.append(df[1][i])
  labels.append(df[2][i])

print(labels)

with open('mod_labels.txt', 'w+') as f:
     
    # write elements of list
    for items in labels:
        f.write('%s\n' %items)

['ZETA', 'EPSILON', 'LAMDA', 'DELTA', 'ALPHA', 'EPSILON', 'RHO', 'PHI', 'PI', 'TAU', 'XI', 'PSI', 'PI', 'OMICRON', 'IOTA', 'DELTA', 'ALPHA', 'CHI', 'ZETA', 'NU', 'OMEGA', 'OMEGA', 'PHI', 'ZETA', 'ALPHA', 'NU', 'CHI', 'OMICRON', 'ETA', 'LAMDA', 'KAPPA', 'DELTA', 'PI', 'KAPPA', 'MU', 'OMICRON', 'ETA', 'THETA', 'XI', 'TAU', 'RHO', 'EPSILON', 'RHO', 'TAU', 'KAPPA', 'BETA', 'OMICRON', 'ETA', 'LAMDA', 'EPSILON', 'PI', 'IOTA', 'EPSILON', 'OMEGA', 'RHO', 'ALPHA', 'UPSILON', 'ALPHA', 'EPSILON', 'PSI', 'XI', 'OMICRON', 'LAMDA', 'PI', 'ZETA', 'GAMMA', 'PSI', 'XI', 'ALPHA', 'ZETA', 'XI', 'NU', 'PI', 'CHI', 'THETA', 'ETA', 'IOTA', 'ETA', 'THETA', 'ZETA', 'ALPHA', 'OMICRON', 'IOTA', 'PHI', 'NU', 'XI', 'GAMMA', 'DELTA', 'PI', 'SIGMA', 'PSI', 'PSI', 'PHI', 'KAPPA', 'OMICRON', 'THETA', 'IOTA', 'KAPPA', 'ZETA', 'XI', 'GAMMA', 'PSI', 'CHI', 'IOTA', 'ETA', 'XI', 'NU', 'ZETA', 'ZETA', 'OMICRON', 'UPSILON', 'MU', 'TAU', 'TAU', 'MU', 'DELTA', 'CHI', 'GAMMA', 'BETA', 'KAPPA', 'OMEGA', 'IOTA', 'PI', 'KAPPA', '

Train CNN

(150, 500, 3)


In [ ]:
#

In [13]:
with open( "train/labels.txt", 'r' ) as file:
	gold_labels = file.read().splitlines()
 
print(gold_labels)

['ZETA,EPSILON,LAMDA', 'DELTA,ALPHA,EPSILON', 'RHO,PHI,PI', 'TAU,XI,PSI', 'PI,OMICRON,IOTA', 'DELTA,ALPHA,CHI', 'ZETA,NU,OMEGA', 'OMEGA,PHI,ZETA', 'ALPHA,NU,CHI', 'OMICRON,ETA,LAMDA', 'KAPPA,DELTA,PI', 'KAPPA,MU,OMICRON', 'ETA,THETA,XI', 'TAU,RHO,EPSILON', 'RHO,TAU,KAPPA', 'BETA,OMICRON,ETA', 'LAMDA,EPSILON,PI', 'IOTA,EPSILON,OMEGA', 'RHO,ALPHA,UPSILON', 'ALPHA,EPSILON,PSI', 'XI,OMICRON,LAMDA', 'PI,ZETA,GAMMA', 'PSI,XI,ALPHA', 'ZETA,XI,NU', 'PI,CHI,THETA', 'ETA,IOTA,ETA', 'THETA,ZETA,ALPHA', 'OMICRON,IOTA,PHI', 'NU,XI,GAMMA', 'DELTA,PI,SIGMA', 'PSI,PSI,PHI', 'KAPPA,OMICRON,THETA', 'IOTA,KAPPA,ZETA', 'XI,GAMMA,PSI', 'CHI,IOTA,ETA', 'XI,NU,ZETA', 'ZETA,OMICRON,UPSILON', 'MU,TAU,TAU', 'MU,DELTA,CHI', 'GAMMA,BETA,KAPPA', 'OMEGA,IOTA,PI', 'KAPPA,OMEGA,ZETA', 'UPSILON,KAPPA,TAU', 'EPSILON,ZETA,PHI', 'OMEGA,CHI,TAU', 'UPSILON,ZETA,NU', 'OMICRON,SIGMA,NU', 'CHI,LAMDA,ZETA', 'DELTA,KAPPA,PSI', 'RHO,TAU,TAU', 'TAU,OMEGA,XI', 'PSI,MU,IOTA', 'DELTA,THETA,CHI', 'PSI,CHI,LAMDA', 'ETA,LAMDA,ALPHA', '